# Solar Energy Consumption and Production Analysis
This notebook analyzes solar energy consumption and production. It calculates various metrics, including PV production, unmet demand, and performance indicators, under different conditions (e.g., number of panels and batteries).

## 1. Importing Modules and Setting Up Environment
In this step, we import necessary modules, set up the working directory, and load the custom script for analysis functions.

In [9]:
# Import necessary modules and custom script
import os
import sys
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import solar_analysis_script as sas

# Set the directory explicitly if needed
#s.chdir(r"....\Milestone1 code")

# Add the current directory to sys.path so Python knows where to find the module
#sys.path.append(r"....\Milestone1 code")


## 2. Loading Data and Setting Parameters
We load the building consumption data, set parameters like the panel and battery options, and configure widgets for user interaction.


In [10]:
# Define parameters
file_path = "Ense3buildingconsumption.csv"
start_date = '2023-09-25 00:00:00'
end_date = '2024-09-25 00:00:00'

# Load and filter data using the functions from the custom script
filtered_data = sas.load_data(file_path, start_date, end_date)

# Panel and Battery options
panel_options = [500, 600, 700, 800, 900]


Loading data from file:  Ense3buildingconsumption.csv
Data successfully loaded and filtered.


f:\Master Study\Study Material\M2 FME Data\Arbaz Data\Green\Codes for project-20241121T133218Z-001\Codes for project\Milestone1 code\solar_analysis_script.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

f:\Master Study\Study Material\M2 FME Data\Arbaz Data\Green\Codes for project-20241121T133218Z-001\Codes for project\Milestone1 code\solar_analysis_script.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## 3. Widgets for User Interaction
We create interactive sliders and dropdowns for selecting the number of panels, batteries, and the specific month for which analysis is conducted.


In [ ]:
# Widgets for user interaction
num_panels_slider = widgets.IntSlider(
    value=500, 
    min=500, 
    max=1500, 
    step=100, 
    description='Num Panels:', 
    style={'description_width': 'initial'},
    continuous_update=False
)

# Update battery slider to allow values from 0 to 60
num_batteries_slider = widgets.IntSlider(
    value=0, 
    min=0, 
    max=100, 
    step=1, 
    description='Num Batteries:', 
    style={'description_width': 'initial'},
    continuous_update=False
)

month_dropdown = widgets.Dropdown(
    options=[(pd.to_datetime(month, format='%m').strftime('%B'), month) for month in range(1, 13)],
    value=1,
    description='Month:',
    style={'description_width': 'initial'}
)

# Output widget to show the highest consumption day
highest_consumption_output = widgets.Output()

# Display the widgets
display(num_panels_slider, num_batteries_slider, month_dropdown, highest_consumption_output)

def print_highest_consumption_day(change=None):
    """Print the day with the highest consumption for the selected month."""
    with highest_consumption_output:
        highest_consumption_output.clear_output()  # Clear previous output
        
        # Fetch user input for the selected month
        month = month_dropdown.value
        
        # Filter data for the selected month
        monthly_data = filtered_data[filtered_data.index.month == month]

        if monthly_data.empty:
            print(f"[Warning] No data available for the selected month: {month}")
            return

        # Calculate total daily consumption for each day in the month and find the day with the highest consumption
        daily_consumption = monthly_data.resample('D').sum()
        max_consumption_day = daily_consumption['Consumption'].idxmax()

        # Print the date of the day with the highest consumption
        print(f"The day with the highest consumption for {pd.to_datetime(month, format='%m').strftime('%B')} is: {max_consumption_day.date()}")

def update_graph(change=None):
    """Update the graph based on user input."""
    # Fetch user inputs
    num_panels = num_panels_slider.value
    num_batteries = num_batteries_slider.value
    month = month_dropdown.value

    # Filter data for the selected month
    monthly_data = filtered_data[filtered_data.index.month == month]

    if monthly_data.empty:
        print(f"[Warning] No data available for the selected month: {month}")
        return

    # Calculate total daily consumption for each day in the month and find the day with the highest consumption
    daily_consumption = monthly_data.resample('D').sum()
    max_consumption_day = daily_consumption['Consumption'].idxmax()

    # Get the original hourly data for the day with the highest consumption
    peak_day_data = monthly_data[monthly_data.index.date == max_consumption_day.date()]

    # Ensure peak_day_data contains all hours (0-23)
    peak_hourly_load = peak_day_data.set_index(peak_day_data.index.hour)

    # Calculate PV production for the specified month and number of panels
    monthly_pv_production = sas.calculate_monthly_pv_production(month, num_panels, sas.panel_capacity)

    # Battery logic: calculate unmet demand, battery charging, discharging, and excess energy
    battery_soc = sas.initial_soc  # Start with initial state of charge (SOC)
    battery_capacity = sas.battery_capacity_per_unit * num_batteries  # Calculate total battery capacity
    energy_discharged = []
    energy_charged = []
    unmet_demand = []
    excess_energy = []

    for hour in range(24):
        # Ensure the hour exists in the peak_hourly_load; if not, set consumption to 0
        if hour not in peak_hourly_load.index:
            consumption = 0
        else:
            consumption = peak_hourly_load.loc[hour]['Consumption']

        pv_production = monthly_pv_production[hour]

        if pv_production >= consumption:
            # Excess energy available
            excess = pv_production - consumption

            if num_batteries > 0 and battery_soc < 100:
                # Charge the battery only if there's capacity
                charge = min(excess * sas.converter_efficiency, (100 - battery_soc) / 100 * battery_capacity)
                energy_charged.append(charge)
                battery_soc += (charge / battery_capacity) * 100
                excess_energy.append(-(excess - charge))  # Excess energy after charging shown as negative value
            else:
                energy_charged.append(0)
                excess_energy.append(-excess)  # All excess is "excess energy" if no battery, shown as negative value

            energy_discharged.append(0)
            unmet_demand.append(0)
        else:
            # PV production is not enough to meet demand
            deficit = consumption - pv_production

            if num_batteries > 0 and battery_soc > 20:
                # Discharge the battery if SOC is above the minimum threshold
                discharge = min(deficit, (battery_soc - 20) / 100 * battery_capacity)
                energy_discharged.append(discharge)
                battery_soc -= (discharge / battery_capacity) * 100
                unmet_demand.append(deficit - discharge)
            else:
                energy_discharged.append(0)
                unmet_demand.append(deficit)
            energy_charged.append(0)
            excess_energy.append(0)

    # Plot Solar PV Production, Unmet Demand, Energy Charged, Energy Discharged, and Electricity Demand
    fig = go.Figure()

    # Add bar traces for different components
    fig.add_trace(go.Bar(x=np.arange(24), y=monthly_pv_production, name='Solar PV Production (kW)', marker_color='orange'))
    fig.add_trace(go.Bar(x=np.arange(24), y=unmet_demand, name='Unmet Demand (kW)', marker_color='red'))

    # If batteries are used, add energy charged and discharged traces
    if num_batteries > 0:
        fig.add_trace(go.Bar(x=np.arange(24), y=[-val for val in energy_charged], name='Energy Charged to Battery (kW)', marker_color='green'))
        fig.add_trace(go.Bar(x=np.arange(24), y=energy_discharged, name='Energy Discharged from Battery (kW)', marker_color='purple'))

    # Add excess energy trace (energy that wasn't stored or used, shown below the x-axis)
    fig.add_trace(go.Bar(x=np.arange(24), y=excess_energy, name='Excess Energy (kW)', marker_color='cyan'))

    fig.add_trace(go.Scatter(x=np.arange(24), y=peak_hourly_load['Consumption'], mode='lines', name='Electricity Demand (kW)', line=dict(color='blue', dash='dash')))

    # Update layout
    fig.update_layout(
        title=f'Solar PV Production, Unmet Demand, Energy Charged, Energy Discharged, Excess Energy, and Electricity Demand - {pd.to_datetime(month, format="%m").strftime("%B")} with {num_panels} Panels and {num_batteries} Batteries',
        xaxis_title='Hour of the Day',
        yaxis_title='Energy (kWh)',
        xaxis=dict(tickvals=list(range(0, 24)), ticktext=[f'{h}:00' for h in range(0, 24)]),
        template='plotly_white',
        barmode='relative'
    )

    # Clear previous output and display updated figure
    clear_output(wait=True)
    display(num_panels_slider, num_batteries_slider, month_dropdown, highest_consumption_output)
    fig.show()

# Attach the update functions to the interactive widgets
num_panels_slider.observe(update_graph, names='value')
num_batteries_slider.observe(update_graph, names='value')
month_dropdown.observe(update_graph, names='value')
month_dropdown.observe(print_highest_consumption_day, names='value')

# Initial outputs 
print_highest_consumption_day()
update_graph()

IntSlider(value=500, continuous_update=False, description='Num Panels:', max=1500, min=500, step=100, style=Sl…

IntSlider(value=0, continuous_update=False, description='Num Batteries:', style=SliderStyle(description_width=…

Dropdown(description='Month:', options=(('January', 1), ('February', 2), ('March', 3), ('April', 4), ('May', 5…

Output()

# Solar Energy Consumption and Production Analysis 

#### 1. Performance Indicator vs Different number of Panels 
#### 2. Performance Indicator vs Different number of Batteries


In [12]:
# Panel options and number of batteries options
panel_options = [500, 600, 700, 800, 900]
battery_options = [0, 10, 30, 60, 100]  # Number of batteries to evaluate

# Function to calculate the performance indicator for the scenario with different numbers of batteries
def calculate_performance_indicator(num_panels, filtered_data, num_batteries):
    """
    Calculate performance indicator for a given number of batteries.
    
    Parameters:
    - num_panels: Number of solar panels installed.
    - filtered_data: Yearly filtered data of building consumption.
    - num_batteries: Number of batteries installed.

    Returns:
    - performance_indicator: Ratio of locally consumed PV energy to the total PV production.
    """
    total_pv_production_used = 0
    total_battery_energy_used = 0
    annual_pv_production = 0
    battery_soc = sas.initial_soc  # Start with initial state of charge (SOC)
    battery_capacity = sas.battery_capacity_per_unit * num_batteries  # Total battery capacity in kWh

    # Calculate the PV production per month and accumulate total PV production and total used
    for month in range(1, 13):
        pv_production_series = sas.calculate_monthly_pv_production(month, num_panels, sas.panel_capacity)
        monthly_data = filtered_data[filtered_data.index.month == month]

        for timestamp, row in monthly_data.iterrows():
            consumption = row['Consumption']
            pv_production = pv_production_series[timestamp.hour]
            annual_pv_production += pv_production

            if pv_production >= consumption:
                # PV is more than enough to meet demand
                excess_energy = pv_production - consumption
                total_pv_production_used += consumption

                # Charge the battery if there's any excess energy and SOC is below 100%
                if num_batteries > 0 and battery_soc < 100:
                    charge_energy = min(excess_energy * sas.converter_efficiency, (100 - battery_soc) / 100 * battery_capacity)
                    battery_soc += (charge_energy / battery_capacity) * 100  # Update SOC
            else:
                # PV is not enough to meet demand
                deficit = consumption - pv_production
                total_pv_production_used += pv_production

                # Use battery to fulfill the remaining load if SOC is above 20%
                if num_batteries > 0 and battery_soc > 20:
                    discharge_energy = min(deficit, (battery_soc - 20) / 100 * battery_capacity)
                    total_battery_energy_used += discharge_energy
                    battery_soc -= (discharge_energy / battery_capacity) * 100  # Update SOC

    # Performance Indicator: (PV used locally + Battery supplied) / Total PV production
    performance_indicator = (total_pv_production_used + total_battery_energy_used) / annual_pv_production if annual_pv_production > 0 else 0
    return performance_indicator

# Calculate the Performance Indicator for each number of panels with different battery capacities
performance_indicators = {b: [] for b in battery_options}

for num_batteries in battery_options:
    for num_panels in panel_options:
        performance_indicator = calculate_performance_indicator(num_panels, filtered_data, num_batteries)
        performance_indicators[num_batteries].append(performance_indicator)

# Generate plot of Performance Indicator vs Number of Panels for different battery capacities
fig = go.Figure()

# Add line traces for different numbers of batteries
for num_batteries in battery_options:
    fig.add_trace(go.Scatter(
        x=panel_options,
        y=performance_indicators[num_batteries],
        mode='lines+markers',
        name=f'{num_batteries} Batteries',
        line=dict(dash='solid' if num_batteries == 0 else 'dash')
    ))

# Update plot layout
fig.update_layout(
    title='Performance Indicator vs Number of Panels Installed (Different Battery Capacities)',
    xaxis_title='Number of Panels Installed',
    yaxis_title='Performance Indicator (Share of Produced Energy Consumed Locally)',
    height=600,
    xaxis=dict(tickvals=panel_options),
    yaxis=dict(range=[0, 1], tickformat='.2f'),
    template='plotly_white'
)

# Show plot
fig.show()

# Generate plot of Performance Indicator vs Number of Batteries for different numbers of panels
fig3 = go.Figure()

# Calculate Performance Indicator vs Number of Batteries for each number of panels
for num_panels in panel_options:
    performance_indicators_vs_batteries = []

    for num_batteries in battery_options:
        performance_indicator = calculate_performance_indicator(num_panels, filtered_data, num_batteries)
        performance_indicators_vs_batteries.append(performance_indicator)

    # Add line trace for performance indicator vs number of batteries for each number of panels
    fig3.add_trace(go.Scatter(
        x=battery_options,
        y=performance_indicators_vs_batteries,
        mode='lines+markers',
        name=f'Performance Indicator with {num_panels} Panels',
        line=dict(dash='solid')
    ))

# Update plot layout
fig3.update_layout(
    title='Performance Indicator vs Number of Batteries (Different Numbers of Panels Installed)',
    xaxis_title='Number of Batteries Installed',
    yaxis_title='Performance Indicator (Share of Produced Energy Consumed Locally)',
    height=600,
    xaxis=dict(tickvals=battery_options),
    yaxis=dict(range=[0, 1], tickformat='.2f'),
    template='plotly_white'
)

# Show plot
fig3.show()


Calculating monthly PV production for month: 1 with 500 panels and panel capacity 610W
Calculating monthly PV production for month: 2 with 500 panels and panel capacity 610W
Calculating monthly PV production for month: 3 with 500 panels and panel capacity 610W
Calculating monthly PV production for month: 4 with 500 panels and panel capacity 610W
Calculating monthly PV production for month: 5 with 500 panels and panel capacity 610W
Calculating monthly PV production for month: 6 with 500 panels and panel capacity 610W
Calculating monthly PV production for month: 7 with 500 panels and panel capacity 610W
Calculating monthly PV production for month: 8 with 500 panels and panel capacity 610W
Calculating monthly PV production for month: 9 with 500 panels and panel capacity 610W
Calculating monthly PV production for month: 10 with 500 panels and panel capacity 610W
Calculating monthly PV production for month: 11 with 500 panels and panel capacity 610W
Calculating monthly PV production for mon

Calculating monthly PV production for month: 1 with 500 panels and panel capacity 610W
Calculating monthly PV production for month: 2 with 500 panels and panel capacity 610W
Calculating monthly PV production for month: 3 with 500 panels and panel capacity 610W
Calculating monthly PV production for month: 4 with 500 panels and panel capacity 610W
Calculating monthly PV production for month: 5 with 500 panels and panel capacity 610W
Calculating monthly PV production for month: 6 with 500 panels and panel capacity 610W
Calculating monthly PV production for month: 7 with 500 panels and panel capacity 610W
Calculating monthly PV production for month: 8 with 500 panels and panel capacity 610W
Calculating monthly PV production for month: 9 with 500 panels and panel capacity 610W
Calculating monthly PV production for month: 10 with 500 panels and panel capacity 610W
Calculating monthly PV production for month: 11 with 500 panels and panel capacity 610W
Calculating monthly PV production for mon